# Hamiltonian moments of wavefunctions

In order to employ the Gram-Charlier or Edgeworth method of approximating the energy distribution associated with the state $\ket{\psi}$, the moments of the Hamiltonian $H$ with respect to this distribution need to be computed. In Overlapper, we provide two approaches to computing these moments: directly by computing the matrix product of the Hamiltonian and the wavefunction as arrays, and computing the same product using DMRG. 

In this notebook, we showcase both approaches, and use the first (exact) method to benchmark the DMRG-based method in the case of a small molecule.

#### Create the system

Start by creating a PySCF `Molecule` and running an RHF calculation on it

In [1]:
import numpy as np
from pyscf import gto

from overlapper.state import do_hf, hf_state

############### define a system of interest ##############
hftype = "rhf" 
R = 1.
N = 6
mol = gto.M(atom=[['H', (0, 0, i * R)] for i in range(N)], basis='sto6g')

# run  HF calculation
rhf, rhf_e, rhf_ss, rhf_sz = do_hf(mol, hftype)
wf_rhf = hf_state(rhf)
print(f"HF wavefunction: \n {wf_rhf}")
print(f"\nRHF energy: {rhf_e[0]:.3f}")
print(f"\nRHF S^2 and Sz: {rhf_ss[0]:.3f}, {rhf_sz[0]:.3f}")

/home/chiffa/miniconda3/envs/overlapper_v0.0.1/lib/python3.9/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


HF wavefunction: 
 {(7, 7): 1.0}

RHF energy: -3.156

RHF S^2 and Sz: 0.000, 0.000


#### Moments: exact calculation

In principle, given a state $|\psi\rangle$ expressed as a vector ina  finite-dimensional Hilbert space, it is posible to compute any moment $\langle \psi | H^n | \psi \rangle$ by simply executing a series of matrix multiplications. This is exactly what the direct method in Overlapper implements, exploiting PySCF's data structures to accomplish the task.

In [2]:
from overlapper.moments import rfci_moments
kmax = 6
rhf_moments = rfci_moments(rhf, kmax, wf_rhf)
print(rhf_moments)

[-7.75984266e+00  6.03377390e+01 -4.69939671e+02  3.66507164e+03
 -2.86159249e+04  2.23633317e+05]


The calculation of moments for unrestricted Hartree-Fock calculations may be done similarly using the `ufci_moments` method. This could be required when the spin of the molecule is not zero.

In [3]:
from overlapper.moments import ufci_moments
mol_spin2 = gto.M(atom=[['H', (0, 0, i * R)] for i in range(N)], basis='sto6g', spin=2)
uhf, uhf_e, uhf_ss, uhf_sz = do_hf(mol_spin2, "uhf")
wf_uhf = hf_state(uhf)
uhf_moments = ufci_moments(uhf, kmax, wf_uhf)
print(uhf_moments)

[-7.62285303e+00  5.81815416e+01 -4.44529530e+02  3.39925428e+03
 -2.60118466e+04  1.99164983e+05]


The limitation of this matrix-multiplication-based approach is that only very small systems can be handled in this way, with only at most 8-10 spatial orbitals.

#### Moments: via DMRG

Alternatively, the same matrix multiplication can be carried out using the language of matrix-product states and matrix-product operators (MPS and MPO). While the results are not exact, varying in accuracy depending on the value of the bond dimension used in the calculation, this method allows calculation of approximate moments for systems with more than 50-70 spatial orbitals.

As mentioned in notebook 2, the `do_dmrg()` method is special in Overlapper, because DMRG structures are exploited beyond just obtaining an initial state -- for example, for computing moments. 

For this purpose, passing `return_objects = True` returns, instead of the usual initial state, internal runtime objects such as `DMRGDriver`, `mps` of the state and `mpo` of the Hamiltonian. These objects can then be used to calculate moments -- and later on, to execute the resolvent method of approximating the energy distribution.

To make the moments from both methods comparable, a constant energy shift countebalancing the nuclear energy needs to be applied to the DMRG calculation -- accompished below through the `eshift` variable.

In [4]:
from overlapper.state import do_dmrg, dmrg_state
bond_dims = [1]
n_sweeps = [20]
noises = [1e-6]
thrds = [1e-5]
sch = [bond_dims, n_sweeps, noises, thrds]
ncas, nelecas = 6, (3,3)
eshift = -mol.energy_nuc()
mpo, mps, driver = do_dmrg(rhf, ncas, nelecas, sch, eshift=eshift, return_objects=True)

In order to use DMRG to evaluate the moments of a given state, the state must be supplied in MPS format. In this case, since we are after the Hartree-Fock state, we can easily generate this state by running a DMRG calculation at bond dimension 1. If we were after a different state, we could instead employ the converter functionality (example 7) and convert any state in the Overlapper sparse dictionary (SOS) format into the MPS format this way.

We can easily check that we indeed got the Hartree-Fock state, with a short extra step from Block2's `driver` functionality

In [5]:
wf = driver.get_csf_coefficients(mps, cutoff=1e-3, iprint=0)
wf_dmrg = dmrg_state([wf], tol=1e-3)
print(wf_dmrg)

{(7, 7): -1.0}


With the state of interest, the Hamiltonian MPO and the driver in hand, the moments can now be calculated. 

To compute a moment $\langle \psi | H^n | \psi \rangle$, the driver's `multiply()` method is first used to calculate $| \psi_1 \rangle = H| \psi \rangle$, then the first moment is obtained by calculating the expectation value `driver.expectation(ket1, mpo, mps)`. Each subsequent moment is obtained from the previous one recursively by continually applying H to the state: because of this, the bond dimension of the MPS storing each subsequent ket (the "bra") needs to be increased as the calculation proceeds, to ensure improved accuracy.

In [6]:
from overlapper.moments import dmrg_moments
bra_bond_dims = [50 * ii for ii in range(1, kmax+1)]
hf_moments = dmrg_moments(driver, mps, mpo, bra_bond_dims, kmax, verbose=0)
print(hf_moments)

[-7.75984266e+00  6.02722412e+01 -4.68476222e+02  3.64325086e+03
 -2.83444824e+04  2.20862989e+05]


In [7]:
print(rhf_moments)

[-7.75984266e+00  6.03377390e+01 -4.69939671e+02  3.66507164e+03
 -2.86159249e+04  2.23633317e+05]


Very good agreement is seen between computing the moments using both methods.